In [5]:
##
##Script to get List of categories available in Yelp that list restraurants in ranking
##
import pandas as pd
import numpy as np
#for creating geo coordinate combinations
import itertools  as it
from yelpapi import YelpAPI
import requests     # how python goes onto the internet!
import re           # regex
from bs4 import BeautifulSoup # a python HTML parser (version 3)
from urllib2 import Request, urlopen
import json
from pandas.io.json import json_normalize
%matplotlib inline

#### https://www.yelp.com/developers/v2/manage_api_keys refresh to generate new keys

In [6]:
api_key = 'SD5ybcet2QSwWXz2N7M5lw'
api_secret = 'EJBXJBAFgEka0br1ynT6HwWJVT8'
access_token = 'EESADq5kSfBL4XkbxzwJxsqhZIgQHbJs'
access_secret = 'JMU-eh8i0eVUUYxJcjelLkZEfTY'

#### activate yelpapi with personal keys

In [7]:
yelp_api = YelpAPI(api_key, api_secret, access_token, access_secret)

#### Build index of all the field we care about

In [8]:
index = {'id':'True','is_closed':'True','name':'True','image_url':'True', 'url':'True','mobile_url':'True','phone':'True', 'display_phone':'True', 'review_count':'True','categories':'True', 'rating':'True', 'snippet_text':'True', 'location_address':'True', 'location_display_address':'True', 'location_city':'True','location_state_code':'True','location_postal_code':'True','location_country_code':'True','location_cross_streets':'True','location_cross_streets':'True','location_neighborhoods':'True','location_coordinate':'True'}
index

{'categories': 'True',
 'display_phone': 'True',
 'id': 'True',
 'image_url': 'True',
 'is_closed': 'True',
 'location_address': 'True',
 'location_city': 'True',
 'location_coordinate': 'True',
 'location_country_code': 'True',
 'location_cross_streets': 'True',
 'location_display_address': 'True',
 'location_neighborhoods': 'True',
 'location_postal_code': 'True',
 'location_state_code': 'True',
 'mobile_url': 'True',
 'name': 'True',
 'phone': 'True',
 'rating': 'True',
 'review_count': 'True',
 'snippet_text': 'True',
 'url': 'True'}

## Create a method to grab yelp business with a keyword in a lat-long bound

In [9]:
#Create method to return json to row
def parse_restaurant(rest_json):
    index = {'id':'True','is_closed':'True','name':'True','image_url':'True', 'url':'True','mobile_url':'True','phone':'True', 'display_phone':'True', 'review_count':'True','categories':'True', 'rating':'True', 'snippet_text':'True', 'location_address':'True', 'location_display_address':'True', 'location_city':'True','location_state_code':'True','location_postal_code':'True','location_country_code':'True','location_cross_streets':'True','location_neighborhoods':'True','location_coordinate':'True'}
    if 'id' in index: 
        _id = rest_json.get('id')
    else: _id = 'NaN'
    if 'is_closed' in index: 
        is_closed = rest_json.get('is_closed')
    else: is_closed = 'NaN'
    if 'name' in index:
        name = rest_json.get('name')
    else: name = 'NaN'
    if 'image_url' in index:
        image_url = rest_json.get('image_url')
    else: image_url = 'NaN'
    if 'url' in index:
        url = rest_json.get('url')
    else: url = 'NaN'
    if 'mobile_url' in index:
        mobile_url = rest_json.get('mobile_url')
    else: mobile_url = 'NaN'
    if 'phone' in index:
        phone = rest_json.get('phone')
    else: phone = 'NaN'
    if 'display_phone' in index:
        display_phone = rest_json.get('display_phone')
    else: display_phone = 'NaN'
    if 'review_count' in index:
        review_count = rest_json.get('review_count')
    else: review_count = 'NaN'
    if 'categories' in index:
        categories = rest_json.get('categories')
    else: categories = 'NaN'
    if 'rating' in index:
        rating = rest_json.get('rating')
    else: rating = 'NaN'
    if 'snippet_text' in index:
        snippet_text = rest_json.get('snippet_text')
    else: snippet_text = 'NaN'
    if 'location_address' in index:
        location_address = rest_json['location'].get('address')
    else: location_address = 'NaN'
    if 'location_display_address' in index:
        location_display_address = rest_json['location'].get('display_address')
    else: location_display_address = 'NaN'
    if 'location_city' in index:
        location_city = rest_json['location'].get('city')
    else: location_city = 'NaN'
    if 'location_state_code' in index:
        location_state_code = rest_json['location'].get('state_code')
    else: location_state_code= 'Nan'        
    if 'location_postal_code' in index:
        location_postal_code = rest_json['location'].get('postal_code')
    else: location_postal_code = 'Nan'
    if 'location_country_code' in index:
        location_country_code = rest_json['location'].get('country_code')
    else: location_country_code = 'Nan'        
    if 'location_cross_streets' in index:
        location_cross_streets = rest_json['location'].get('cross_streets')
    else: location_cross_streets = 'Nan'
    if 'location_neighborhoods' in index:
        location_neighborhoods = rest_json['location'].get('neighborhoods')
    else: location_neighborhoods = 'Nan'
    if 'location_coordinate' in index:
        location_coordinate = rest_json['location'].get('coordinate')  
    else: location_coordinate= 'Nan'
    #good thing about get function is that it returns if there is a result and doesn't break if there isn't
    return [_id,is_closed, name,image_url,url,mobile_url,phone,display_phone,review_count,categories,rating,snippet_text,location_address, location_display_address, location_city,location_state_code,location_postal_code,location_country_code,location_cross_streets,location_neighborhoods,location_coordinate]

In [10]:
#Create method to call yelp api with term and bound filters
def getbiz(term, bounds):
    search_results_df = pd.DataFrame({})
    offset = 0
    while offset<= 980:
        temp_dict = yelp_api.search_query(term= term, bounds=bounds, offset=offset)
        rows_of_df = []
        for bar_json in temp_dict['businesses']:
            new_row = parse_restaurant(bar_json)
            rows_of_df.append(new_row)
        temp_df = pd.DataFrame(rows_of_df, columns=['id','is_closed','name','image_url', 'url','mobile_url','phone', 'display_phone', 'review_count','categories', 'rating', 'snippet_text', 'location_address', 'location_display_address', 'location_city','location_state_code','location_postal_code','location_country_code','location_cross_streets','location_neighborhoods','location_coordinate'])
        search_results_df = search_results_df.append(temp_df, ignore_index=True)
        if temp_df.shape[0]<20:
            print 'Results exhausted at '+ str(temp_df.shape[0]+offset)+' results at offset '+str(offset)
            break;
        offset += 20
    return search_results_df;

In [11]:
#Function to get 1 if a number is not a multiple of something 
import math
def rem(x,y):
    res = x % y
    return math.copysign(res,x)


In [50]:
#create method to get list of bounds

def get_bounds(nw_latitude, nw_longitude, se_latitude, se_longitude,block):
    #Specify how many rows and columns you wish to devide the geographical bounds into
    #block = 15
    increment_lat = (nw_latitude-se_latitude)/block
    increment_lon = (nw_longitude-se_longitude)/block

    #Find latitude and longitude blocks coordinates
    lat = np.arange(se_latitude, nw_latitude, increment_lat)
    lon = np.arange(se_longitude, nw_longitude, increment_lon)

    #build a block squared list of coordinates
    import itertools
    listoflist = lat,lon

    #make all combinations of bounds required for calculation
    comb  = list(itertools.product(*listoflist))

    #sequence the list of coordinates to run north (east to west) to south (east to west)
    comb = comb[::-1]
    
    i=0
    bounds = []
    #print (block*(block-1)-1)
    while i<=(block*(block-1)-1):
    #while i<=block*block:
        if rem(i,block) != (block-1):
            bounds = bounds + [str(comb[i][0])+','+str(comb[i][1])+'|'+str(comb[i+1+block][0])+','+str(comb[i+1+block][1])]
            #print len(bounds)
            i += 1
        elif rem(i,block) == (block-1):
            i += 1
    return bounds;
    


In [51]:
def getall(term, nw_latitude, nw_longitude, se_latitude, se_longitude, block):
    bounds = get_bounds(nw_latitude, nw_longitude, se_latitude, se_longitude, block)
    #Run getbiz method for each element in bounds
    b = 0
    biz=pd.DataFrame({})
    while b <= (len(bounds)-1):
        biz = biz.append(getbiz(term,bounds[b]), ignore_index=True)
        #print b
        b+=1
    return biz;

In [ ]:
#Specify the keyword to search
term = 'wine'

#Specify the lat lon to specify bounds
nw_latitude=37.810298
nw_longitude=-122.522450
se_latitude=37.704093
se_longitude=-122.375803
block = 5

#Get bounds list using method
#bounds = get_bounds(nw_latitude, nw_longitude, se_latitude, se_longitude, block)

#Get
wine = getall(term,nw_latitude, nw_longitude, se_latitude, se_longitude, block)

Results exhausted at 159 results at offset 140
Results exhausted at 159 results at offset 140
Results exhausted at 499 results at offset 480
Results exhausted at 199 results at offset 180
Results exhausted at 79 results at offset 60
Results exhausted at 79 results at offset 60
Results exhausted at 399 results at offset 380
Results exhausted at 59 results at offset 40
Results exhausted at 86 results at offset 80


Results exhausted at 119 results at offset 100
Results exhausted at 249 results at offset 240
Results exhausted at 499 results at offset 480
Results exhausted at 219 results at offset 200
Results exhausted at 110 results at offset 100
Results exhausted at 79 results at offset 60
Results exhausted at 379 results at offset 360
Results exhausted at 59 results at offset 40


KeyboardInterrupt: 

In [9]:
#Specify the keyword to search
term = 'wine'

#Specify the lat lon to specify bounds
nw_latitude=37.810298
nw_longitude=-122.522450
se_latitude=37.704093
se_longitude=-122.375803
block = 15

#Get bounds list using method
bounds = get_bounds(nw_latitude, nw_longitude, se_latitude, se_longitude, block)

#Run getbiz method for each element in bounds
b = 0
biz=pd.DataFrame({})
while b <= (len(bounds)-1):
    biz = biz.append(getbiz(term,bounds[b]), ignore_index=True)
    #print b
    b+=1
print b
biz.shape

Results exhausted at 0 results at offset 0
Results exhausted at 0 results at offset 0
Results exhausted at 0 results at offset 0
Results exhausted at 1 results at offset 0
Results exhausted at 1 results at offset 0
Results exhausted at 10 results at offset 0
Results exhausted at 8 results at offset 0
Results exhausted at 106 results at offset 100
Results exhausted at 41 results at offset 40
Results exhausted at 49 results at offset 40
Results exhausted at 176 results at offset 160
Results exhausted at 39 results at offset 20
Results exhausted at 2 results at offset 0
Results exhausted at 0 results at offset 0
Results exhausted at 0 results at offset 0
Results exhausted at 0 results at offset 0
Results exhausted at 1 results at offset 0
Results exhausted at 1 results at offset 0
Results exhausted at 3 results at offset 0
Results exhausted at 2 results at offset 0
Results exhausted at 1 results at offset 0
Results exhausted at 3 results at offset 0
Results exhausted at 14 results at offs

(3730, 21)

In [11]:
biz.head()

,id,is_closed,name,image_url,url,mobile_url,phone,display_phone,review_count,categories,...,snippet_text,location_address,location_display_address,location_city,location_state_code,location_postal_code,location_country_code,location_cross_streets,location_neighborhoods,location_coordinate
0,baker-beach-sand-ladder-san-francisco-2,False,Baker Beach Sand Ladder,https://s3-media2.fl.yelpcdn.com/bphoto/N7LaQl...,https://www.yelp.com/biz/baker-beach-sand-ladd...,https://m.yelp.com/biz/baker-beach-sand-ladder...,None,None,27.0,"[[Local Flavor, localflavor]]",...,Did you know about the Sand Ladder? If you di...,"[Baker Beach, Lincoln Blvd]","[Baker Beach, Lincoln Blvd, Presidio, San Fran...",San Francisco,CA,94129,US,None,[Presidio],"{u'latitude': 37.7967685093, u'longitude': -12..."
1,doughbies-san-francisco,False,Doughbies,https://s3-media2.fl.yelpcdn.com/bphoto/1dl5h7...,https://www.yelp.com/biz/doughbies-san-francis...,https://m.yelp.com/biz/doughbies-san-francisco...,2016769602,+1-201-676-9602,186.0,"[[Bakeries, bakeries], [Food Delivery Services...",...,I felt compelled to write a review because the...,[],"[Financial District, San Francisco, CA 94104]",San Francisco,CA,94104,US,None,[Financial District],"{u'latitude': 37.7915267944, u'longitude': -12..."
2,he-commissary-san-francisco,False,The Commissary,https://s3-media4.fl.yelpcdn.com/bphoto/srMW70...,https://www.yelp.com/biz/he-commissary-san-fra...,https://m.yelp.com/biz/he-commissary-san-franc...,4155613600,+1-415-561-3600,257.0,"[[Spanish, spanish], [American (New), newameri...",...,My friends and I had a recent dinner club at T...,[101 Montgomery St],"[101 Montgomery St, Presidio, San Francisco, C...",San Francisco,CA,94129,US,None,[Presidio],"{u'latitude': 37.799758721, u'longitude': -122..."
3,american-lamb-jam-san-francisco,False,American Lamb Jam,https://s3-media3.fl.yelpcdn.com/bphoto/YMo-s1...,https://www.yelp.com/biz/american-lamb-jam-san...,https://m.yelp.com/biz/american-lamb-jam-san-f...,None,None,19.0,"[[Specialty Food, gourmet], [Wineries, wineries]]",...,Just got home from the second year in a row at...,[135 Fisher Lp],"[135 Fisher Lp, Presidio, San Francisco, CA 94...",San Francisco,CA,94123,US,None,[Presidio],"{u'latitude': 37.7999573, u'longitude': -122.4..."
4,transit-san-francisco-2,False,TRANSIT,https://s3-media4.fl.yelpcdn.com/bphoto/yvt4m6...,https://www.yelp.com/biz/transit-san-francisco...,https://m.yelp.com/biz/transit-san-francisco-2...,4155614435,+1-415-561-4435,56.0,"[[Cafes, cafes], [Coffee & Tea, coffee], [Amer...",...,"man, what a gem. a partial, but great view of...",[215 Lincoln Blvd],"[215 Lincoln Blvd, Presidio, San Francisco, CA...",San Francisco,CA,94129,US,Ralston Ave & Merchant Rd,[Presidio],"{u'latitude': 37.8015004, u'longitude': -122.4..."


In [562]:
#Export to csv
#Previous = pd.read_csv('../data/yelp_businesses.csv', parse_dates=True)
biz.to_csv('../data/yelp_businesses.csv',encoding='utf-8')

# debugging code:

In [568]:
#Create a list of all bounds to pass through getbiz() method

i=0
bounds = []
#print (block*(block-1)-1)
while i<=(block*(block-1)-1):
    if rem(i,block) != (block-1):
        bounds = bounds + [str(comb[i][0])+','+str(comb[i][1])+'|'+str(comb[i+1+block][0])+','+str(comb[i+1+block][1])]
        #print len(bounds)
        i += 1
    elif rem(i,block) == (block-1):
        i += 1
        #print 'this is where im at'+str(i)

In [15]:
from pprint import pprint

In [513]:
biz = yelp_api.search_query(term= 'bar', bounds=bounds[0])

In [514]:
pprint(biz['businesses'][0]['image_url'])

IndexError: list index out of range

In [515]:
pprint(biz['total'])

0


In [496]:
biz['businesses'].
rest_json['id'].get(index,'')

SyntaxError: invalid syntax (<ipython-input-496-38c3826c8558>, line 1)

In [256]:
biz['businesses'].get('phone')

AttributeError: 'list' object has no attribute 'get'

In [396]:
index = {'id':'True','is_closed':'True','name':'True'} #,'image_url', 'url','mobile_url','phone', 'display_phone', 'review_count','categories', 'rating', 'snippet_text', 'location_address', 'location_display_address', 'location_city','location_state_code','location_postal_code','location_country_code','location_cross_streets','location_cross_streets','location_neighborhoods','location_coordinate'}
if 'id' in index: 
    print biz.get('id')

0    mangiare-san-francisco
Name: id, dtype: object


In [397]:
def parse_restaurant(rest_json): 
    if 'id' in index: 
        _id = rest_json['id']
    else _id = 'NaN'
    if 'is_closed' in index: 
        is_closed = rest_json['is_closed']
    if 'name' in index:
        name = rest_json['name']
    image_url = rest_json['image_url']
    url = rest_json['url']
    mobile_url = rest_json['mobile_url']
    phone = rest_json['phone']
    #rest_json['phone']
    display_phone = rest_json['display_phone']
    #rest_json['display_phone']
    review_count = rest_json['review_count']
    categories = rest_json['categories']
    rating = rest_json['rating']
    snippet_text = rest_json['snippet_text']
    location_address = rest_json['location'].get('address')
    location_display_address = rest_json['location'].get('display_address')[1]
    location_city = rest_json['location'].get('city')
    location_state_code = rest_json['location'].get('state_code')
    location_postal_code = rest_json['location'].get('postal_code')
    location_country_code = rest_json['location'].get('country_code')
    location_cross_streets = rest_json['location'].get('cross_streets')
    location_neighborhoods = rest_json['location'].get('neighborhoods')
    location_coordinate = rest_json['location'].get('coordinate')  
    #good thing about get function is that it returns if there is a result and doesn't break if there isn't
    return [_id,is_closed, name,image_url,url,mobile_url,phone,display_phone,review_count,categories,rating,snippet_text,location_address, location_display_address, location_city,location_state_code,location_postal_code,location_country_code,location_cross_streets,location_cross_streets,location_neighborhoods,location_coordinate]

In [471]:
pprint(biz)

{u'businesses': [{u'categories': [[u'Recreation Centers', u'recreation'],
                                  [u'Soccer', u'football']],
                  u'display_phone': u'+1-415-561-4200',
                  u'id': u'fort-scott-field-at-the-presidio-san-francisco',
                  u'image_url': u'https://s3-media4.fl.yelpcdn.com/bphoto/LZVhxPq2TpTpq4AGFd8jHg/ms.jpg',
                  u'is_claimed': True,
                  u'is_closed': False,
                  u'location': {u'address': [],
                                u'city': u'San Francisco',
                                u'coordinate': {u'latitude': 37.8031542208,
                                                u'longitude': -122.474578895},
                                u'country_code': u'US',
                                u'display_address': [u'Presidio',
                                                     u'San Francisco, CA 94129'],
                                u'geo_accuracy': 9.5,
                             

In [516]:
# Sinan's code
rows_of_df = []
for bar_json in biz['businesses']:
    new_row = parse_restaurant(bar_json)
    rows_of_df.append(new_row)

In [517]:
# Sinan's code
pd.DataFrame(rows_of_df, columns=['id','is_closed','name','image_url', 'url','mobile_url','phone', 'display_phone', 'review_count','categories', 'rating', 'snippet_text', 'location_address', 'location_display_address', 'location_city','location_state_code','location_postal_code','location_country_code','location_cross_streets','location_cross_streets','location_neighborhoods','location_coordinate'])

,id,is_closed,name,image_url,url,mobile_url,phone,display_phone,review_count,categories,...,location_address,location_display_address,location_city,location_state_code,location_postal_code,location_country_code,location_cross_streets,location_cross_streets,location_neighborhoods,location_coordinate


In [59]:
# Sinan's code
def parse_restaurant(rest_json):
    _id = rest_json['id']
    name = rest_json['name']
    location = rest_json['location'].get('address')[0]
    #good thing about get function is that it returns if there is a result and doesn't break if there isn't
    return [_id, name, location]
    

In [141]:
pprint(biz['businesses'][2])

{u'categories': [[u'Cocktail Bars', u'cocktailbars'],
                 [u'Lounges', u'lounges']],
 u'display_phone': u'+1-415-757-0290',
 u'id': u'topsys-fun-house-san-francisco',
 u'image_url': u'https://s3-media1.fl.yelpcdn.com/bphoto/rIThdpt70JhgOufkK5jArg/ms.jpg',
 u'is_claimed': True,
 u'is_closed': False,
 u'location': {u'address': [u'260 Kearny St'],
               u'city': u'San Francisco',
               u'coordinate': {u'latitude': 37.7905828,
                               u'longitude': -122.4038175},
               u'country_code': u'US',
               u'cross_streets': u'Bush St & Hardie Pl',
               u'display_address': [u'260 Kearny St',
                                    u'Financial District',
                                    u'San Francisco, CA 94108'],
               u'geo_accuracy': 9.5,
               u'neighborhoods': [u'Financial District'],
               u'postal_code': u'94108',
               u'state_code': u'CA'},
 u'mobile_url': u'https://m.yelp.c

In [38]:
#george's code
tweets = []
for i in tweepy(api):
    e = []
    e.append(tweet["text"])
    e.append(tweet["entities"]["hashtags"])
    tweets.append(e)

NameError: name 'tweepy' is not defined